---
title: "Taxonomic Classification"
editor: visual
jupyter: python3
---


# Generating a taxonomic file

We perform taxonomic classification using [Kraken2](https://benlangmead.github.io/aws-indexes/k2), a tool that assigns reads based on k-mer matches (short nucleotide sequences of a defined length). Taxonomic classification is a hierarchical system that organizes organisms into increasingly specific groups based on shared characteristics and evolutionary relationships.

This step involves a few commands. Lets break it up.

## Make sure you are in the right folder.

You should have already created the "kraken2_standard" folder in the Preparation chapter. 

``` bash
cd /mnt/viro0002-data/sequencedata/processed/Diagnostics_metagenomics/Viro_Run_0001/barcode01/kraken2_standard/ 
```

::: callout-warning
## Warning

Remember to update the run paths and barcodes for each sample!
:::

## Kraken2

Next, you need to specify the location of the Kraken2 database, the input reads (e.g., all_reads_QC_hg19.fastq), and the desired output file along with its destination.

``` bash
cd /mnt/viro0002-data/sequencedata/processed/Diagnostics_metagenomics/Viro_Run_0001/barcode01/
kraken2 --db /mnt/viro0002-data/workgroups_projects/Bioinformatics/DB/Kraken2/Standard --use-names --report-zero-counts --threads 16 --report report_standard.txt --output assignments.txt all_reads_QC_hg19.fastq
```

🔹<strong style="color:darkblue">--use-names</strong> – Outputs taxonomic names instead of just numeric taxon IDs, making the report more readable.\
🔹<strong style="color:darkblue">--zero-counts</strong> – Includes taxa that have zero reads assigned in the report. Useful for comprehensive taxonomic overviews.\
🔹<strong style="color:darkblue">--threads 16</strong> – Runs the classification using 16 CPU threads to speed up the analysis.\
🔹<strong style="color:darkblue">--report report.txt</strong> – Generates a detailed, tab-delimited classification report (used for downstream visualization like **Krona** which we will use).\
🔹<strong style="color:darkblue">--output kraken_output.txt</strong> – Outputs individual read classification results, listing each read and its assigned taxon.

To make things easier (and faster), you can search for specific phrases within the assignments file in the Kraken2 output. As we are only interested in virues, we can extract only viral entries.

``` bash
awk '{for (i=3; i<=NF; i++) s = s $i " "; if (s ~ /virus/ || s ~ /viridae/) print; s=""}' assignments.txt > virus_only.txt
```

🔹<strong style="color:darkblue">awk</strong> – A powerful text processing tool used for pattern scanning and processing.\
🔹<strong style="color:darkblue">{for (i=3; i\<=NF; i++) s = s \$i " "; ...}</strong> – A block of instructions.\
🔹<strong style="color:darkblue">if (s \~ /virus/) print;</strong> – screens for the word <strong>"virus"and "Viridae"</strong>. 🔹<strong style="color:darkblue">virus_only.txt</strong> – The new output file containing only lines where the word "virus" was found.

# Screening and confirmation

**(i) Open the virus_only.txt file.**

Copy all text onto a new excel sheet and save as "barcode01" in the Kraken2_standard folder.

::: callout-tip
## Tip

What you will see, typically depends on the sample material. If the run is successful, you will likely see lots of bacteriophages and typical human virome organisms, such as torque teno virus.
:::

In some cases, no viral reads may be generated. This just means that no viral reads could be detected within this sample, possibly due to:

-   not enough sequencing depth\
-   nucleic acid degradation
-   sequencing bias of high abundant background nucleic acid\
-   possible error during the wet lab

This typically occurs in CSF samples, which often have low overall viral abundance. In such cases, the sample can be repeated, particularly if a wet lab error is suspected or if sufficient sequencing depth is achievable. Otherwise, it should be reported as: "No clinically relevant virus detected."

**(ii) Filter based on name in alphabetical order on the excel sheet.**

All detection hits of interest need to be checked as some hits could be false positives and should not be reported. We do this by blasting the read on [NCBI](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PROGRAM=blastn&BLAST_SPEC=GeoBlast&PAGE_TYPE=BlastSearch). If multiple hits of the same organism are identified, copy the longest read name.

![Example](taxa.jpg)

**(iii) Search through your fastq file.**

**grep** can be used to search for lines in a file that match a specified pattern. We are going to use **grep** to search for the read ID. We will then generate a txt file with the fastq sequence. In this example, we will use read ID "f2b7c080-e0f5-421e-8593-8559fbe951fb". Remember to change the name of the pathogen of interest.

``` bash
grep -A 3 "f2b7c080-e0f5-421e-8593-8559fbe951fb" all_reads_QC_hg19.fastq > Name_of_potential_pathogen.txt
```

Open the resulting txt file. Copy and paste the sequence into the Query box on [NCBI](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PROGRAM=blastn&BLAST_SPEC=GeoBlast&PAGE_TYPE=BlastSearch).

**(iv) Examine the NCBI output.**

Check the Percentage Identity, Query Cover and Length. The example below represents the results of blasting an example norovirus read.

![Example](noro.jpg)

::: callout-important
## Important

In this case, we can confirm detection.
:::

**(vi) Update the excel report with your findings for each sample.**

Depending on the clinical question, more analysis maybe required.

The next chapter "Visualization" will help you make krona plots to illustrate taxonomic classification.

## Interpretation

What is clinically relevant?

Viruses are considered relevant if they have been associated with clinical symptoms consistent with disease.

• Usually a target on current molecular assays\
• Literature\
• Experience (through screening many datasheets of different sample materials)\
• Patient metadata\
• Clinical history (age, immune status etc.)\
• Clinical presentation (and sample material)

We can only report what we find (or what is likely), a multidisciplinary clinical team is still needed to determine final causative agent of the patient’s clinical presentation

A few examples have been provided to help with interpretation.

![Example 1](QC_importance1.jpg)

::: callout-important
## Important

This detection could not be confirmed and will not be reported.
:::

![Example 2](QC_importance2.jpg)

::: callout-important
## Important

This detection could not be confirmed and will not be reported.
:::

------------------------------------------------------------------------